# This script shows how to get data from Wikipedia and clean it.

In [83]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
#!pip install geopy 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
!pip install folium
import folium
import json
import requests
from pandas.io.json import json_normalize

!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data
geosp_data_frame = pd.read_csv('Geospatial_data.csv')

URL transformed to HTTPS due to an HSTS policy
--2021-03-16 15:34:38--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 52.116.127.82, 52.116.121.148
Connecting to cocl.us (cocl.us)|52.116.127.82|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-03-16 15:34:41--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-03-16 15:34:41--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9

Get data from the link and retreave the coumns:

In [84]:
                                                                                                            
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'


dfs = pd.read_html(url)
df = dfs[0]                                                                                                        
data_frame = df[['Postal Code','Borough','Neighbourhood']]


Clean the data in the set:

In [85]:
data_frame.drop(data_frame[data_frame['Borough'] == 'Not assigned'].index, inplace = True) 
data_frame.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Merge the dataframes:

In [87]:
df_merge = data_frame.merge(geosp_data_frame, on="Postal Code", how = 'inner')
df_merge.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


I decided to plot the ma of toronto and query data from Foursquare.

In [88]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer_test")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [89]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_merge['Latitude'],df_merge['Longitude'],df_merge['Borough'],df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto


In [90]:


CLIENT_ID = '4AQWJX5KU2Z1N45M3TNVKEKACLPF5E3YINDJCOZU5CLHNVKD' # your Foursquare ID
CLIENT_SECRET = 'LDY3T4DOSDUF0H4J3JS2UFES2XAEPHZPS5U0D1CSGV152YUI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


Your credentails:
CLIENT_ID: 4AQWJX5KU2Z1N45M3TNVKEKACLPF5E3YINDJCOZU5CLHNVKD
CLIENT_SECRET:LDY3T4DOSDUF0H4J3JS2UFES2XAEPHZPS5U0D1CSGV152YUI


In [91]:


neighbourhood_latitude = df_merge.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = df_merge.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = df_merge.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [93]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6050d1e1a9fe7f66508f5147'},
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'distance': 245,
        'cc': 'CA'